# Imports

In [1]:
import os
import numpy as np

from tqdm import tqdm

import matplotlib.pyplot as plt

In [2]:
from Nim.Nim import Nim
from Nim.NimLogic import NimLogic

from Agents.MCTS.MCTSAgentV1 import MCTSAgentV1

Random seed for reproducibility

In [3]:
np.random.seed(42)

# Agent Testing Function

In [4]:
def test_agent(_misereAgent, _normalAgent, _misere, _initial_piles):
    explored_nodes = np.ndarray(EPISODES)
    moves_count = np.ndarray(EPISODES)
    mean_nodes = np.ndarray(EPISODES)

    for i in tqdm(range(EPISODES)):
        game = Nim(
            initial_piles=_initial_piles[i],
            misere=_misere[i]
        )

        agent = _misereAgent if _misere[i] else _normalAgent

        winner = game.play(
            player1=agent,
            player2=agent,
            verbose=False
        )

        assert winner == NimLogic.is_p_position(_initial_piles[i], _misere[i])

        agent.compute_mean_nodes()

        mean_nodes[i] = agent.mean_nodes
        moves_count[i] = agent.moves_count
        explored_nodes[i] = agent.nodes_explored

    weighted_mean_nodes = explored_nodes.sum() / moves_count.sum()

    avg_explored_nodes = explored_nodes.mean()
    avg_moves_count = moves_count.mean()

    values = [weighted_mean_nodes, avg_explored_nodes, avg_moves_count]

    labels = [
        "average explored nodes per move:",
        "average explored nodes per game:",
        "average moves per game:"
    ]

    label_width = max(len(lbl) for lbl in labels)

    for lbl, val in zip(labels, values):
        print(f"{lbl:<{label_width}} {val:>10.2f}")

    return explored_nodes, moves_count, mean_nodes

# Test running function

In [5]:
def run_tests(misere_v1, misere_v2, misere_v3, normal_v1, normal_v2, normal_v3, max_depth, pile_count, max_pile, episodes, save_path_prefix):
    os.makedirs(save_path_prefix, exist_ok=True)

    print("-" * 60)
    classic_configuration = False

    if pile_count == 0 or max_pile == 0:
        classic_configuration = True
        initial_piles = np.tile([1, 3, 5, 7], (episodes, 1))
        print(f"\tTestare fara max_depth, pe configuratia clasica")
    else:
        initial_piles = np.random.randint(1, max_pile, size=(episodes, pile_count))
        print(f"\tTestare cu max_depth={max_depth}, pile_count={pile_count}, max_pile={max_pile}")

    print("-" * 60)
    misere_modes = np.random.choice([True, False], size=episodes)

    results = {
        'v1': test_agent(misere_v1, normal_v1, misere_modes, initial_piles),
        'v2': test_agent(misere_v2, normal_v2, misere_modes, initial_piles),
        'v3': test_agent(misere_v3, normal_v3, misere_modes, initial_piles),
    }

    for version, result in results.items():
        if not classic_configuration:
            filename = f"{save_path_prefix}/minimax-{version}-{max_depth}-{pile_count}-{max_pile}.npz"
        else:
            filename = f"{save_path_prefix}/minimax-{version}-{max_depth}-classic.npz"
        np.savez(filename, result)

# Plotting results

In [6]:
def _first_existing_file(patterns):
    for p in patterns:
        if os.path.exists(p):
            return p
    return None

def plot_results(vary_parameter,
                 max_depth_values,
                 vary_values,
                 fixed_pile_count,
                 fixed_max_pile,
                 folder="savedData/Minimax",
                 save_folder="savedPlots/Minimax"):
    os.makedirs(save_folder, exist_ok=True)

    models       = ['v1', 'v2', 'v3']
    model_names  = ['Minimax V1', 'Minimax V2', 'Minimax V3']
    colors       = ['#4B0082', '#4169E1', '#9370DB']
    metric_labels = [
        'Avg Nodes/Move',
        'Avg Nodes/Game',
        'Avg Moves/Game'
    ]

    bar_width = 0.3
    x_indices = np.arange(len(vary_values))

    for max_depth in max_depth_values:
        fig, axes = plt.subplots(1, 3, figsize=(10, 4), facecolor='white', sharex=True)
        axes = axes.ravel()

        all_metrics = {m: {i: [] for i in range(3)} for m in models}

        for vary_value in vary_values:
            for model in models:
                pile_count = vary_value if vary_parameter == "heap_count" else fixed_pile_count
                max_pile   = fixed_max_pile if vary_parameter == "heap_count" else vary_value

                candidate_files = [
                    f"{folder}/minimax-{model}-{max_depth}-{pile_count}-{max_pile}.npz",
                    # f"{folder}/minimax-{model}-{max_depth}-classic.npz",
                ]

                filename = _first_existing_file(candidate_files)

                if filename is None:
                    print(f"⚠️ Lipsă fișier pentru: depth={max_depth}, "
                          f"{vary_parameter}={vary_value} → {candidate_files[-1].split('/')[-1]}")
                    for i in range(3):
                        all_metrics[model][i].append(np.nan)
                    continue

                data                = np.load(filename)
                test_data           = data.get('arr_0')
                explored_nodes, moves_count, mean_nodes = test_data

                metrics_values = [
                    explored_nodes.sum() / moves_count.sum(),
                    explored_nodes.mean(),
                    moves_count.mean()
                ]
                for i in range(3):
                    all_metrics[model][i].append(metrics_values[i])

        for metric_idx, ax in enumerate(axes):
            for model_idx, model in enumerate(models):
                offset = -0.2 + 0.2 * model_idx
                ax.bar(
                    x_indices + offset,
                    all_metrics[model][metric_idx],
                    width=bar_width,
                    color=colors[model_idx],
                    alpha=1,
                    # alpha=0.6 + 0.2 * model_idx,
                    # edgecolor='black',
                    # linewidth=0.8,
                    label=model_names[model_idx]
                )
            ax.set_ylabel(metric_labels[metric_idx], fontsize=14)
            ax.set_xlabel(vary_parameter.upper(), fontsize=14)
            ax.set_xticks(x_indices)
            ax.set_xticklabels(vary_values)
            ax.grid(True, axis='y', alpha=0.3)
            if metric_idx == 0:
                ax.legend(loc='upper right')

        plt.tight_layout(rect=[0, 0, 1, 0.97])

        save_path = os.path.join(save_folder,
                                 f"minimax_variation_{vary_parameter}_depth_{max_depth}.png")
        plt.savefig(save_path, facecolor='white')
        plt.close()


In [7]:
EPISODES = 10000

PILE_COUNT = 4
MAX_PILE = 127

for _ in range(EPISODES):
    initial_piles = np.random.randint(1, MAX_PILE, size=(PILE_COUNT,))
    misere = np.random.choice([True, False], PILE_COUNT)

    normalAgent = MCTSAgentV1(
        misere=False,
        simulation_limit=1000,
        time_limit=1.0
    )

    misereAgent = MCTSAgentV1(
        misere=True,
        simulation_limit=1000,
        time_limit=1.0
    )

    test_agent(
        misereAgent,
        normalAgent,
        misere,
        initial_piles
    )

  0%|          | 0/10000 [00:00<?, ?it/s]


TypeError: 'numpy.int64' object is not iterable